In [3]:
import os
import pandas as pd
import numpy as np

def motion_detection(rssi_buffer, rssi_index, rssi_buffer_filled, RSSI_BUFFER_SIZE=20):
    if not rssi_buffer_filled and rssi_index < 5:
        return False
    count = RSSI_BUFFER_SIZE if rssi_buffer_filled else rssi_index
    mean = sum(rssi_buffer[:count]) / count
    variance = sum((x - mean) ** 2 for x in rssi_buffer[:count]) / count
    return variance

def calculate_file_variances(file_path, RSSI_BUFFER_SIZE=20):
    df = pd.read_csv(file_path)
    rssi_values = df['rssi'].values
    
    variances = []
    rssi_buffer = np.zeros(RSSI_BUFFER_SIZE)
    rssi_index = 0
    rssi_buffer_filled = False
    
    for rssi in rssi_values:
        rssi_buffer[rssi_index] = rssi
        rssi_index += 1
        
        if rssi_index >= RSSI_BUFFER_SIZE:
            rssi_buffer_filled = True
            rssi_index = 0
        
        variance = motion_detection(rssi_buffer, rssi_index, rssi_buffer_filled, RSSI_BUFFER_SIZE)
        if variance is not False:
            variances.append(variance)
    
    return variances

def classify_file(file_path, threshold, RSSI_BUFFER_SIZE=20):
    variances = calculate_file_variances(file_path)
    mean_variance = np.mean(variances)
    
    if mean_variance > threshold:
        print(f"File {os.path.basename(file_path)}: Motion (1), Mean Variance: {mean_variance:.2f}")
        return 1
    else:
        print(f"File {os.path.basename(file_path)}: Static (0), Mean Variance: {mean_variance:.2f}")
        return 0

motion_files = [os.path.join("./evaluation_motion", file) for file in os.listdir("./evaluation_motion") if file.endswith(".csv")]
static_files = [os.path.join("./evaluation_static", file) for file in os.listdir("./evaluation_static") if file.endswith(".csv")]
test_files = [os.path.join("./test", file) for file in os.listdir("./test") if file.endswith(".csv")]
motion_files.sort()
static_files.sort()
test_files.sort()

motion_variances = []
static_variances = []

for file in motion_files:
    variances = calculate_file_variances(file)
    motion_variances.extend(variances)

for file in static_files:
    variances = calculate_file_variances(file)
    static_variances.extend(variances)

# motion_mean = np.mean(motion_variances)
# motion_std = np.std(motion_variances)
# static_mean = np.mean(static_variances)
# static_std = np.std(static_variances)
threshold = 0.5 # (motion_mean + static_mean) / 2

print(f"Threshold: {threshold:.2f}")
# print(f"Motion mean variance: {motion_mean:.2f}, std: {motion_std:.2f}")
# print(f"Static mean variance: {static_mean:.2f}, std: {static_std:.2f}")

print("\nClassifying motion files:")
for file in motion_files:
    classify_file(file, threshold)

print("\nClassifying static files:")
for file in static_files:
    classify_file(file, threshold)

print("\nClassifying test files:")
for file in test_files:
    classify_file(file, threshold)

Threshold: 0.50

Classifying motion files:
File CSI_20250220_204932.csv: Motion (1), Mean Variance: 10.44
File CSI_20250220_205301.csv: Motion (1), Mean Variance: 6.40
File CSI_20250220_205318.csv: Motion (1), Mean Variance: 15.00
File CSI_20250220_205330.csv: Motion (1), Mean Variance: 11.87
File CSI_20250220_205343.csv: Motion (1), Mean Variance: 8.71
File CSI_20250220_205356.csv: Motion (1), Mean Variance: 6.61
File CSI_20250220_205406.csv: Motion (1), Mean Variance: 8.26
File CSI_20250220_205439.csv: Motion (1), Mean Variance: 11.06
File CSI_20250220_205451.csv: Motion (1), Mean Variance: 9.19
File CSI_20250220_205504.csv: Motion (1), Mean Variance: 10.59
File CSI_20250220_205516.csv: Motion (1), Mean Variance: 11.04
File CSI_20250220_205526.csv: Motion (1), Mean Variance: 10.24
File CSI_20250220_205539.csv: Motion (1), Mean Variance: 12.32
File CSI_20250220_205553.csv: Motion (1), Mean Variance: 9.58
File CSI_20250220_205607.csv: Motion (1), Mean Variance: 3.17
File CSI_20250220_2